In [ ]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# from models import Full_Record
from db_conn import engine, session_scope, ping_db, Base
from models import FactFunding, DimensionOrganization, DimensionLocation, DimensionCountry, DimensionContinent, DimensionInvestor, InvestorMapping, DimensionIndustry, IndustryMapping, FactNews

from eda_and_preprocessing.preprocessing_class import Preprocessing

In [25]:
ping_db(engine)

True

In [ ]:
data = Preprocessing()

In [12]:
# Drop the existing table and create the database table according to the defined model
# Base.metadata.drop_all(engine)
# Base.metadata.create_all(engine)

In [ ]:
# data.locations_df --> DimensionLocation
with session_scope() as session:
    # Step 1: Insert or Fetch Continent Records
    continents = {}
    for continent_name in data.locations_df['Continent'].unique():
        continent = session.query(DimensionContinent).filter_by(continent_name=continent_name).first()
        if not continent:
            continent = DimensionContinent(continent_name=continent_name)
            session.add(continent)
            session.flush()  # Generate `continent_id` without committing
        continents[continent_name] = continent.continent_id

    # Step 2: Insert or Fetch Country Records
    countries = {}
    for _, row in data.locations_df[['Country', 'Continent']].drop_duplicates().iterrows():
        country_name = row['Country']
        continent_id = continents[row['Continent']]
        country = session.query(DimensionCountry).filter_by(country_name=country_name).first()
        if not country:
            country = DimensionCountry(country_name=country_name, continent_id=continent_id)
            session.add(country)
            session.flush()  # Generate `country_id` without committing
        countries[country_name] = country.country_id

    # Step 3: Populate DimensionLocation with Foreign Keys
    for _, row in data.locations_df.iterrows():
        country_id = countries[row['Country']]
        location = DimensionLocation(
            location_id=row['LocationID'],
            city=row['City'],
            country_id=country_id,
            longitude=row['Longitude'],
            latitude=row['Latitude']
        )
        session.add(location)

    # Commit all changes
    session.commit()

In [ ]:
# data.organizations_df --> DimensionOrganization
with session_scope() as session:
    for _, row in data.organizations_df.iterrows():
        location = DimensionOrganization(
        organization_id=row['OrganizationID'],
        organization_name=row['Organization Name'],
        location_id=row['LocationID'],
        organization_name_url=row['Organization Name URL'],
        organization_description=row['Organization Description'],
        organization_website=row['Organization Website']
        )
        session.add(location)
    session.commit()

In [7]:
# data.df --> FactFunding
with session_scope() as session:
    for _, row in data.df.iterrows():
        equity_only = row['Equity Only Funding'].lower() == 'true'
        funding = FactFunding(
            transaction_id=row['TransactionID'],
            organization_id=row['OrganizationID'],
            money_raised=row['Money Raised'],
            money_raised_usd=row['Money Raised (in USD)'],
            funding_type=row['Funding Type'],
            announced_date=row['Announced Date'],
            funding_stage=row['Funding Stage'],
            number_of_funding_rounds=row['Number of Funding Rounds'],
            total_funding_amount=row['Total Funding Amount'],
            total_funding_amount_usd=row['Total Funding Amount (in USD)'],
            equity_only=equity_only,
            transaction_name=row['Transaction Name'],
            transaction_name_url=row['Transaction Name URL'],
            money_raised_currency=row['Money Raised Currency'],
            pre_money_valuation=row['Pre-Money Valuation'],
            pre_money_valuation_currency=row['Pre-Money Valuation Currency'],
            pre_money_valuation_usd=row['Pre-Money Valuation (in USD)'],
            funding_status=row['Funding Status'],
            number_of_investors=row['Number of Investors']
        )
        session.add(funding)
    session.commit()

In [8]:
# data.investors_df --> DimensionInvestor
with session_scope() as session:
    for _, row in data.investors_df.iterrows():
        investor = DimensionInvestor(
            investor_id=row['InvestorID'],
            investor=row['Investor']
        )
        session.add(investor)
    session.commit()

In [9]:
# data.investor_mapping_df --> InvestorMapping
with session_scope() as session:
    for _, row in data.investor_mapping_df.iterrows():
        investor_mapping = InvestorMapping(
            transaction_id=int(row['TransactionID']),
            investor_id=int(row['InvestorID']),
            is_lead_investor=bool(row['IsLeadInvestor'])
        )
        session.add(investor_mapping)
    session.commit()

In [10]:
# data.industries_df --> DimensionIndustry
with session_scope() as session:
    for _, row in data.industries_df.iterrows():
        industry = DimensionIndustry(
            industry_id=row['IndustryID'],
            industry=row['Industry']
        )
        session.add(industry)
    session.commit()

In [11]:
# data.industry_mapping_df --> IndustryMapping
with session_scope() as session:
    for _, row in data.industry_mapping_df.iterrows():
        industry_mapping = IndustryMapping(
            organization_id=int(row['OrganizationID']),
            industry_id=int(row['IndustryID'])
        )
        session.add(industry_mapping)
    session.commit()

### Query Example:


In [15]:
# 1. Query all funding transactions
with session_scope() as session:
    all_fundings = session.query(FactFunding).all()
    for funding in all_fundings:
        print(f"Transaction ID: {funding.transaction_id}, Money Raised: {funding.money_raised}, Funding Type: {funding.funding_type}")

Transaction ID: 1, Money Raised: 2800000.0, Funding Type: Seed
Transaction ID: 2, Money Raised: 632000.0, Funding Type: Seed
Transaction ID: 3, Money Raised: nan, Funding Type: Seed
Transaction ID: 4, Money Raised: 23500000.0, Funding Type: Seed
Transaction ID: 5, Money Raised: 21956027.0, Funding Type: Pre-Seed
Transaction ID: 6, Money Raised: 25000.0, Funding Type: Seed
Transaction ID: 7, Money Raised: nan, Funding Type: Pre-Seed
Transaction ID: 8, Money Raised: nan, Funding Type: Seed
Transaction ID: 9, Money Raised: nan, Funding Type: Seed
Transaction ID: 10, Money Raised: nan, Funding Type: Pre-Seed
Transaction ID: 11, Money Raised: nan, Funding Type: Seed
Transaction ID: 12, Money Raised: 500000.0, Funding Type: Pre-Seed
Transaction ID: 13, Money Raised: 20000.0, Funding Type: Pre-Seed
Transaction ID: 14, Money Raised: nan, Funding Type: Seed
Transaction ID: 15, Money Raised: 5487000.0, Funding Type: Seed
Transaction ID: 16, Money Raised: nan, Funding Type: Seed
Transaction ID: 1

In [16]:
# 2. Query organizations located in a specific country
with session_scope() as session:
    organizations_in_denmark = session.query(DimensionOrganization).join(DimensionLocation).filter(DimensionLocation.country == 'Denmark').all()
    for org in organizations_in_denmark:
        print(f"Organization Name: {org.organization_name}, Country: Denmark")

Organization Name: Vuuh, Country: Denmark
Organization Name: FLEXeCHARGE, Country: Denmark
Organization Name: Frankly Insure, Country: Denmark
Organization Name: Mash Makes, Country: Denmark
Organization Name: GetWhy A/S, Country: Denmark
Organization Name: Invert, Country: Denmark
Organization Name: Impactly, Country: Denmark
Organization Name: IDUN Audio, Country: Denmark
Organization Name: Judico, Country: Denmark
Organization Name: THE UPCYCL, Country: Denmark
Organization Name: Doubble, Country: Denmark
Organization Name: Dryp, Country: Denmark
Organization Name: Leneo, Country: Denmark
Organization Name: Coalescent Mobile Robotics, Country: Denmark
Organization Name: Provenance Tags, Country: Denmark
Organization Name: Naetion, Country: Denmark
Organization Name: INSTRAT360, Country: Denmark
Organization Name: Factbird ApS, Country: Denmark
Organization Name: Softpay, Country: Denmark
Organization Name: Fullview, Country: Denmark
Organization Name: Zupply, Country: Denmark
Organi

In [17]:
# 3. Query all investors for a specific transaction
with session_scope() as session:
    transaction_id = 1 
    investors_for_transaction = session.query(DimensionInvestor).join(InvestorMapping).filter(InvestorMapping.transaction_id == transaction_id).all()
    for investor in investors_for_transaction:
        print(f"Investor ID: {investor.investor_id}, Investor Name: {investor.investor}")

Investor ID: 1, Investor Name: Charles Delingpole
Investor ID: 2, Investor Name: Donald Bringmann
Investor ID: 3, Investor Name: Erik Muttersbach
Investor ID: 4, Investor Name: Four Cities Capital
Investor ID: 5, Investor Name: Fredrik Thomassen
Investor ID: 6, Investor Name: Kevin Costa
Investor ID: 7, Investor Name: Louis Beryl
Investor ID: 8, Investor Name: Moonfire Ventures
Investor ID: 9, Investor Name: Pioneer Fund
Investor ID: 10, Investor Name: Sundeep Ahuja
Investor ID: 11, Investor Name: Y Combinator


In [18]:
# 4. Query funding rounds for organizations in a specific industry
with session_scope() as session:
    industry_name = 'FinTech'  # Example industry
    fundings_in_industry = session.query(FactFunding).join(DimensionOrganization).join(IndustryMapping).join(DimensionIndustry).filter(DimensionIndustry.industry == industry_name).all()
    for funding in fundings_in_industry:
        print(f"Organization ID: {funding.organization_id}, Money Raised: {funding.money_raised_usd}, Industry: {industry_name}")

Organization ID: 1, Money Raised: 2800000.0, Industry: FinTech
Organization ID: 13, Money Raised: 20000.0, Industry: FinTech
Organization ID: 20, Money Raised: 5000000.0, Industry: FinTech
Organization ID: 27, Money Raised: 3956926.0, Industry: FinTech
Organization ID: 29, Money Raised: 2000000.0, Industry: FinTech
Organization ID: 35, Money Raised: 5500000.0, Industry: FinTech
Organization ID: 55, Money Raised: 3233019.0, Industry: FinTech
Organization ID: 85, Money Raised: 3750000.0, Industry: FinTech
Organization ID: 100, Money Raised: 2922393.0, Industry: FinTech
Organization ID: 109, Money Raised: 1700000.0, Industry: FinTech
Organization ID: 122, Money Raised: 250000.0, Industry: FinTech
Organization ID: 133, Money Raised: 6400000.0, Industry: FinTech
Organization ID: 109, Money Raised: nan, Industry: FinTech
Organization ID: 153, Money Raised: 16637293.0, Industry: FinTech
Organization ID: 171, Money Raised: nan, Industry: FinTech
Organization ID: 173, Money Raised: 130000.0, In

In [19]:
# 5. Query lead investors in a specific transaction
with session_scope() as session:
    lead_investors = session.query(DimensionInvestor).join(InvestorMapping).filter(InvestorMapping.transaction_id == transaction_id, InvestorMapping.is_lead_investor == True).all()
    for lead_investor in lead_investors:
        print(f"Lead Investor ID: {lead_investor.investor_id}, Lead Investor Name: {lead_investor.investor}")

Lead Investor ID: 8, Lead Investor Name: Moonfire Ventures
Lead Investor ID: 11, Lead Investor Name: Y Combinator


In [26]:
# 6. Query all news entries
with session_scope() as session:
    all_news = session.query(FactNews).all()
    for news in all_news:
        print(f"News ID: {news.news_id}, Title: {news.title}, Published At: {news.published_at}, Source: {news.source}")


News ID: 1, Title: Inside Flagright: Our Approach to Hiring and Team Growth, Published At: 2024-09-14, Source: prweb.com
News ID: 2, Title: UK climate fintech Zero secures partners ahead of launch, Published At: 2024-08-28, Source: finextra.com
News ID: 3, Title: Cloud Native Digest: Score accepted as a CNCF Sandbox Project, Published At: 2024-08-12, Source: medium.com
News ID: 4, Title: AI: A Game-Changer for SREs and Container Orchestration, Published At: 2024-08-26, Source: medium.com
News ID: 5, Title: Generative AI for Kubernetes: meet K8sGPT open source project, Published At: 2024-06-07, Source: cncf.io
News ID: 6, Title: Berlin-based MYNE raises €40 million to further scale co-ownership of premium vacation homes, Published At: 2024-08-01, Source: eu-startups.com
News ID: 7, Title: GNU must emulate Eskom in turnaround, Published At: 2024-07-09, Source: citizen.co.za
News ID: 8, Title: Digital Holdings Group Champions Renewable Energy in Bitcoin Mining Sector, Published At: 2024-0